In [ ]:
import numpy as np
import pandas as pd
from IPython.display import clear_output
from time import sleep, time 
import re
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
# 操作 browser 的 API
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options 

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException, WebDriverException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

from webdriver_manager.chrome import ChromeDriverManager
# 啟動瀏覽器工具的選項
# my_options = webdriver.ChromeOptions()
# # my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
# #my_options.add_argument("--start-maximized")         #最大化視窗
# my_options.add_argument("--incognito")               #開啟無痕模式
# my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
# my_options.add_argument("--disable-notifications")  #取消 chrome 推播通知
# my_options.add_argument("--lang=zh-TW")  #設定為正體中文

C:\Users\student\anaconda3\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
URL = 'hAttps://www.google.com.tw/maps/@23.546162,120.6402133,8z?hl=zh-TW'

In [3]:
def chromeDriver(name: str, addr: str) -> int:
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()),options=chrome_options)

    driver.get(URL)

    inputElement = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, 'searchboxinput')))
    try:
        inputElement.send_keys(name + " " + addr)
    except :
        return 88

    driver.find_element(By.CSS_SELECTOR,'#searchbox > div.pzfvzf > button.mL3xi').click()
    
    try:
        addr = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button > div.AeaXub > div.rogA2c > div.Io6YTe.fontBodyMedium')))
        if re.search("台北市|新北市", addr.text) is None:
            return 88
    except WebDriverException:
        pass
    
    user_input = ""
    while user_input == "":
        try:
            user_input = int(input("Other:0, Breakfast:1, Fastfood:2, Exotic:3, Taiwan:4, Chinese:5, Japan:6 : ") or "4")
        except ValueError:
            user_input = ""
            
    return user_input

In [4]:
df_class = pd.read_csv("C:/Users/student/Desktop/classifier.csv")

In [5]:
df_none_class = df_class[df_class["新分類"].isna()]
df_none_class.drop(columns=['Unnamed: 0'])

,No,name,style,address,新分類


In [6]:
name_list = df_none_class["name"].tolist()

In [7]:
addr_list = df_none_class["address"].tolist()

In [8]:
# all_class = {"Other":[],"Breakfast":[],"Fastfood":[],"Exotic":[],"Taiwan":[],"Chinese":[],"Japan":[]}

In [9]:
# all_class["Breakfast"] = []

In [10]:
# all_class["Fastfood"] = []

In [11]:
# all_class["Exotic"] = []

In [12]:
# all_class["Taiwan"] = []

In [13]:
# all_class["Japan"] = []

In [14]:
# all_class["Chinese"] = []

In [15]:
# all_class["Other"] = []

In [16]:
def chromeDriver_notInRange(searchKey: str):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    driver.get(URL)

    inputElement = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, 'searchboxinput')))
    inputElement.send_keys(searchKey)

    driver.find_element(By.CSS_SELECTOR,'#searchbox > div.pzfvzf > button.mL3xi').click()
    
    try:
        addr = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button > div.AeaXub > div.rogA2c > div.Io6YTe.fontBodyMedium')))
        if re.search("台北市|新北市", addr.text) is None:
            df_class.loc[df_class["name"] == searchKey, ["新分類"]] = "X"
            print(f'{df_class.loc[df_class["name"] == searchKey, ["新分類"]].index.values} {searchKey} {addr.text}')
    except :
        if df_class.loc[df_class["name"] == searchKey, ["新分類"]] is None:
            df_class.loc[df_class["name"] == searchKey, ["新分類"]] = None
    

In [17]:
# with ThreadPoolExecutor() as executor:
#     executor.map(chromeDriver_notInRange, check_list)

In [18]:
# df_class = pd.read_csv("C:/Users/student/Desktop/classifier2.csv")

In [19]:
# df_class.drop(df_class.index[8457],inplace=True)
# df_class.to_csv("C:/Users/student/Desktop/classifier.csv",index=None)

In [20]:
# df_none_class = df_class[df_class["新分類"].isna()]
# df_none_class.drop(columns=['Unnamed: 0'])

In [21]:
auto_save_count = 0
for name, addr in zip(name_list, addr_list):
    clear_output()
    
    x = chromeDriver(name, addr)
    if x == 0:
        df_class.loc[df_class["name"] == name, ["新分類"]] = "Other"
    elif x == 1:
        df_class.loc[df_class["name"] == name, ["新分類"]] = "Breakfast"
    elif x == 2:
        df_class.loc[df_class["name"] == name, ["新分類"]] = "Fastfood"
    elif x == 3:
        df_class.loc[df_class["name"] == name, ["新分類"]] = "Exotic"
    elif x == 4:
        df_class.loc[df_class["name"] == name, ["新分類"]] = "Taiwan"
    elif x == 5:
        df_class.loc[df_class["name"] == name, ["新分類"]] = "Chinese"
    elif x == 6:
        df_class.loc[df_class["name"] == name, ["新分類"]] = "Japan"
    elif x == 88:
        df_class.loc[df_class["name"] == name, ["新分類"]] = "X"
    elif x == 999:
        df_class.to_csv("C:/Users/student/Desktop/classifier.csv",index=None)
        print("Saved File...")
        break
    else:
        df_class.loc[df_class["name"] == name, ["新分類"]] = None
    
    auto_save_count += 1
    
    if auto_save_count >= 1:
        auto_save_count = 0
        df_class.to_csv("C:/Users/student/Desktop/classifier.csv",index=None)
        print("Auto Saving File...")
#         sleep(3)
